In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [56.8 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-05-16 11:39:36--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-05-16 11:39:36 (6.03 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL_AWS_game").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Configuration for RDS instance
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/big_data_db"
mode_1="append"
config_1 = {"user":"postgres",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}
mode_2="overwrite"
config_2 = {"user":"postgres",
          "password": "<insert password>",
          "truncate": "true",
          "driver":"org.postgresql.Driver"}

In [ ]:
from pyspark import SparkFiles
#load software reviews data from s3.amazonaws.com
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

game_df = spark.read.option("header","true").csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), inferSchema=True, dateFormat="yyyy-MM-dd",sep="\t")
game_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

# **Drop duplicates and incomplete rows**

In [ ]:
print(game_df.count())
game_df = game_df.dropna()
print(game_df.count())
game_df = game_df.dropDuplicates()
print(game_df.count())

145431
145425
145425


# **Examine the schema**

In [ ]:
game_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



# **Transfor Dataframe type**

In [ ]:
import datetime
game_df = game_df.withColumn("review_date",game_df["review_date"].cast("date"))
game_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   13860932|R3D22004JZNQNZ|B009VIKJUS|     892290956|     Xbox Music Pass|Digital_Video_Games|          5|            0|          0|   N|                Y|     Quick and Easy!|Save yourself a t...| 2015-08-31|
|         US|     115394|R3JRQ3N85TXT9P|B00STYMPK2|     621402517|Dragon Ball Xenov...|Digital_Video_Games| 

In [ ]:
game_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



# **Create Dataframe for review_id_table**

In [ ]:
review_id_df = game_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3D22004JZNQNZ|   13860932|B009VIKJUS|     892290956| 2015-08-31|
|R3JRQ3N85TXT9P|     115394|B00STYMPK2|     621402517| 2015-08-24|
|R3HL6P7MERPZXD|   11561103|B00K4S9C9I|     843235170| 2015-08-22|
|R2KG7ZWHPD36O2|   17290108|B004RMK4P8|     384246568| 2015-08-21|
|R1ZNBPJ7NVOUZU|   17216137|B004RMK4BC|     384246568| 2015-08-21|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
# Drop duplicated review existed in sql database
review_id_df_pre = spark.read.jdbc(url=jdbc_url,table="review_id_table", properties=config_1)
review_id_df = review_id_df.join(review_id_df_pre, ["review_id"], "leftanti")
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R106IQ5T0O0VWM|   48087383|B00CX6G1IE|     161335416| 2013-08-08|
|R107U3DTMQDOC9|   45843629|B0085P7G5E|     242628037| 2015-05-06|
| R10P6EEWZ3N7L|   50911990|B003H4QG7C|     821948296| 2010-05-15|
|R10WS97KTREEBO|   14896044|B00HG0B8NA|     860280326| 2014-11-10|
|R114BEBH58ZQH7|   51501861|B007VTVRFA|     402222185| 2013-03-07|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
print(review_id_df.count())
review_id_df = review_id_df.dropDuplicates(["review_id"])
print(review_id_df.count())

145425
145425


# **Create Dataframe for products_table**

In [ ]:
products_df = game_df.select(["product_id", "product_title"])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B009VIKJUS|     Xbox Music Pass|
|B00STYMPK2|Dragon Ball Xenov...|
|B00K4S9C9I|SPINTIRES [Online...|
|B004RMK4P8|Playstation Netwo...|
|B004RMK4BC|Playstation Netwo...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
# Drop duplicated review existed in sql database
products_df_pre = spark.read.jdbc(url=jdbc_url,table="products", properties=config_1)
products_df = products_df.join(products_df_pre, ["product_id"], "leftanti")
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001KC02UA|Happy Hour! [Down...|
|B002FGU06Q|Tradewinds Odysse...|
|B002FGU06Q|Tradewinds Odysse...|
|B002FGU06Q|Tradewinds Odysse...|
|B002FGU06Q|Tradewinds Odysse...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
print(products_df.count())
products_df = products_df.dropDuplicates(["product_id"])
print(products_df.count())

145418
7946


# **Create Dataframe for customers_table**

In [ ]:
customers_df_new = game_df.select(["customer_id"])
customers_df_new = customers_df_new.groupBy("customer_id").count()
customers_df_new.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    1821467|    1|
|   13037028|    1|
|   48214430|    1|
|    6568070|    1|
|   37168977|    3|
+-----------+-----+
only showing top 5 rows



In [ ]:
# Creat a copy of sql database
customers_df_pre = spark.read.jdbc(url=jdbc_url,table="customers", properties=config_1)
schema = customers_df_pre.schema
customers_df_pre_pd = customers_df_pre.toPandas()
_customers_df_pre = spark.createDataFrame(customers_df_pre_pd,schema=schema)
del customers_df_pre_pd

In [ ]:
# Get the total counts of each customer
if _customers_df_pre.count():
  customers_df=customers_df_new.join(_customers_df_pre,on=["customer_id"],how="outer")
  customers_df=customers_df.fillna(0)
  customers_df=customers_df.withColumn('total', sum(customers_df[col] for col in ["count", "customer_count"]))
  customers_df = customers_df.drop("count", "customer_count")
  customers_df = customers_df.withColumnRenamed("total", "customer_count")
  customers_df.show(5)
else:
  customers_df = customers_df_new.withColumnRenamed("count", "customer_count")
  customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|     178254|             1|
|     295048|             4|
|     477371|             1|
|     527768|             1|
|     587604|             1|
+-----------+--------------+
only showing top 5 rows



In [ ]:
print(customers_df.count())

205731


# **Create Dataframe for vine_table**

In [ ]:
vine_df = game_df.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3D22004JZNQNZ|          5|            0|          0|   N|
|R3JRQ3N85TXT9P|          1|            4|          6|   N|
|R3HL6P7MERPZXD|          5|            0|          0|   N|
|R2KG7ZWHPD36O2|          1|            0|          0|   N|
|R1ZNBPJ7NVOUZU|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [ ]:
# Drop duplicated review existed in sql database
vine_df_pre = spark.read.jdbc(url=jdbc_url,table="vine_table", properties=config_1)
vine_df = vine_df.join(vine_df_pre, ["review_id"], "leftanti")
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R106IQ5T0O0VWM|          5|            0|          0|   N|
|R107U3DTMQDOC9|          3|            0|          1|   N|
| R10P6EEWZ3N7L|          4|            5|          5|   N|
|R10WS97KTREEBO|          4|            0|          0|   N|
|R114BEBH58ZQH7|          1|            7|          7|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [ ]:
print(vine_df.count())
vine_df = vine_df.dropDuplicates(["review_id"])
print(vine_df.count())

145425
145425


# **Write DataFrame to RDS**

In [ ]:
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode_1, properties=config_1)

In [ ]:
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode_1, properties=config_1)

In [ ]:
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode_2, properties=config_2)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode_1, properties=config_1)